# Wrangle
- identify the way to combine both 

## Data set up

In [2]:
# imports
from wrangle import get_clean_gdf10, get_clean_gdf20, get_clean_mac
import pandas as pd
import requests

In [3]:
# getting the data
mac = get_clean_mac()
gdf10 = get_clean_gdf10()

Dropping 24008 rows


In [4]:
# separating the 2010 datamac
mac10 = mac[mac['created'] < '2020']

mac10.head()

,keyword,subcategory,incident_address,township,city,zip,council_district,created,last_modified,closed,status,origin,close_time
8010,Animal,Dead,903 N OXFORD ST,CENTER,INDIANAPOLIS,46201,17.0,2019-12-02 17:20:19+00:00,2019-12-03 21:32:36+00:00,2019-12-03 21:32:36+00:00,Closed,Phone,1 days 04:12:17
8011,Abandoned Vehicle,Street/Alley (ABV),11506 LULLSTREAM DR,WARREN,INDIANAPOLIS,46229,14.0,2019-12-02 17:29:46+00:00,2019-12-06 17:30:02+00:00,2019-12-06 17:30:02+00:00,Closed,RequestIndy Mobile,4 days 00:00:16
8012,Abandoned Vehicle,Street/Alley (ABV),338 N DREXEL AVE,CENTER,INDIANAPOLIS,46201,12.0,2019-12-02 17:41:20+00:00,2020-05-19 17:45:35+00:00,2019-12-06 17:44:59+00:00,Closed,RequestIndy Online,4 days 00:03:39
8017,Abandoned Vehicle,Street/Alley (ABV),676 S MITCHNER AVE,WARREN,INDIANAPOLIS,46239,18.0,2019-12-02 01:49:28+00:00,2020-05-19 17:45:35+00:00,2019-12-06 02:00:00+00:00,Closed,RequestIndy Mobile,4 days 00:10:32
8018,Trash,96 Gal Cart - Stolen,436 S GRAY ST,CENTER,INDIANAPOLIS,46201,12.0,2019-12-02 17:11:44+00:00,2019-12-04 13:57:24+00:00,2019-12-04 13:57:24+00:00,Closed,Phone,1 days 20:45:40


## Acessing API Census Geocoding for onelineaddress

In [3]:
# attempting to access the census bureau api

input = {
    'benchmark': 'Public_AR_Current',
    'vintage': 'Census2010_Current',    
    'address': '388 North Drexel Ave, Indianapolis, IN, 46201',
    'format': 'json',
    'layers': 'Census Blocks'
    }

# setting 
returntype = 'geographies'
searchtype = 'onelineaddress'

url = f'https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}'

r = requests.post(url, params=input)

print(r.status_code)

import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

200
{
    "result": {
        "addressMatches": [
            {
                "addressComponents": {
                    "city": "INDIANAPOLIS",
                    "fromAddress": "300",
                    "preDirection": "N",
                    "preQualifier": "",
                    "preType": "",
                    "state": "IN",
                    "streetName": "DREXEL",
                    "suffixDirection": "",
                    "suffixQualifier": "",
                    "suffixType": "AVE",
                    "toAddress": "498",
                    "zip": "46201"
                },
                "coordinates": {
                    "x": -86.09142059738242,
                    "y": 39.77338120375392
                },
                "geographies": {
                    "Census Blocks": [
                        {
                            "ACT": "C3",
                            "AREALAND": 29748,
                            "AREAWATER": 0,
                         

In [83]:
# getting only the geoid
r.json()['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['GEOID']

'180973554003002'

In [6]:
# need to use integers to find the correct location
gdf10[gdf10['blockce10'] == 3002]

# trying for the geoid now
gdf10[gdf10['geoid10'] == 180973554003002]

# its a match


,geoid10,countyfp10,blockce10,tractce10,blockname10,aland10,awater10,shape_length,shape_area
202270,180973554003002,97,3002,355400,3002,29748,0,786.575252,29728.855966


## Getting Census Bureau batch addresses
TODO: keep the state fip code and change it to the abbreviation for indiana in the prepare file for the mac census data

In [5]:
# getting number of unique incident addresses
mac10.incident_address.nunique() / 10_000

# we will need 26 different csv's to get the data, I will try to make this as programmatic as possible
mac10['state'] = 'IN'

unique_address_mac_10 = mac10[['incident_address', 'city','state','zip']].drop_duplicates()

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_2222/3223090854.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mac10['state'] = 'IN'


In [10]:
# start with one batch, then make it programmatic
# separate the first 10,000 and save it as a csv with a unique name

batch_1 = unique_address_mac_10.iloc[:9999]

file_path = './data/address_batch_1.csv'

batch_1.to_csv(file_path, header=False)

In [18]:
col_names = list(batch_1.columns)
pd.read_csv(file_path, names=col_names)

,incident_address,city,state,zip
8010,903 N OXFORD ST,INDIANAPOLIS,IN,46201
8011,11506 LULLSTREAM DR,INDIANAPOLIS,IN,46229
8012,338 N DREXEL AVE,INDIANAPOLIS,IN,46201
8017,676 S MITCHNER AVE,INDIANAPOLIS,IN,46239
8018,436 S GRAY ST,INDIANAPOLIS,IN,46201
...,...,...,...,...
62582,1230 W 31ST ST,INDIANAPOLIS,IN,46208
62583,937 N ARLINGTON AVE,INDIANAPOLIS,IN,46219
62584,1722 N EMERSON AVE,INDIANAPOLIS,IN,46218
62585,4026 CORLISS ST,INDIANAPOLIS,IN,46217


## Accessing the API for batch addresses

In [19]:
# post the request to the api

input = {
    'benchmark': 'Public_AR_Current',
    'vintage': 'Census2010_Current',
    'addressFile': file_path
    }

# setting 
returntype = 'geographies'
searchtype = 'addressbatch'

url = f'https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}'

r = requests.post(url, params=input)

In [20]:
r.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <title>Census Geocoder</title>\r\n  <meta charset="utf-8">\r\n  <meta name="viewport" content="width=device-width, initial-scale=1">\r\n  <link rel="icon" sizes="192x192" href="https://www.census.gov/etc.clientlibs/census/clientlibs/common-site/resources/icons/android-chrome-192x192.png">\r\n  <link rel="icon" sizes="256x256" href="https://www.census.gov/etc.clientlibs/census/clientlibs/common-site/resources/icons/android-chrome-256x256.png">\r\n  <link rel="shortcut icon" sizes="32x32" href="https://www.census.gov/etc.clientlibs/census/clientlibs/common-site/resources/icons/favicon.ico">\r\n  <link rel="apple-touch-icon" sizes="180x180" href="https://www.census.gov/etc.clientlibs/census/clientlibs/common-site/resources/icons/apple-touch-icon-180x180.png"> \r\n  <link rel="stylesheet" href="/geocoder/static/css/bootstrap.min.css">\r\n  <link rel="stylesheet" href="/geocoder/static/css/main.css">\r\n  <script src="/geocoder/static/js/b

In [ ]:
# make the output go to the data folder

# add all unmatched addresses to a new df

# open the file with the information and merge it to the mac 10 dataset using the addressess, cities, state, and zipcode as the joiing section, specifically wiht a left join

## Using the [Census Bureua API tutorial](https://www.youtube.com/watch?v=nIdmmEfW3R0) from youtube


In [3]:
import io
import csv

### trying with 10,000 addresses in the batch

In [ ]:
# start with one batch, then make it programmatic
# separate the first 10,000 and save it as a csv with a unique name

batch_1 = unique_address_mac_10.iloc[:9999]

file_path = './data/address_batch_1.csv'

batch_1.to_csv(file_path, header=False)


# setting url information
returntype = 'geographies'
searchtype = 'addressbatch'
url = f'https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}'

# establishing the 
file_path = './data/address_batch_1.csv'
files = {'addressFile': (file_path, open(file_path, 'rb'), 'text/csv')}


payload = {'benchmark':'Public_AR_Current', 'vintage':'Census2010_Current'}

s = requests.post(url, files=files, data=payload)

# got a connectin timeout error, adjusting the file size

### trying with 5,000 addresses in the batch

In [ ]:
# reducung the size of the csv saved to half and trying again
# start with one batch, then make it programmatic
# separate the first 5,000 and save it as a csv with a unique name

batch_1 = unique_address_mac_10.iloc[:5000]

file_path = './data/address_batch_1.csv'

batch_1.to_csv(file_path, header=False)

NameError: name 'unique_address_mac_10' is not defined

In [49]:
# setting url information
returntype = 'geographies'
searchtype = 'addressbatch'
url = f'https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}'

# establishing the file path and opening it
file_path = './data/address_batch_1.csv'
files = {'addressFile': (file_path, open(file_path, 'rb'), 'text/csv')}

# identifying the data type to pull
payload = {'benchmark':'Public_AR_Current', 'vintage':'Census2010_Current'}

# post the requests
s = requests.get(url, files=files, data=payload)

# another connection timeout

### trying with 3750

In [46]:
# reducung the size of the csv saved to half and trying again
# start with one batch, then make it programmatic
# separate the first 3750 and save it as a csv with a unique name

batch_1 = unique_address_mac_10.iloc[:3750]

file_path = './data/address_batch_1.csv'

batch_1.to_csv(file_path, header=False)

requests.models.Response

In [ ]:
# setting url information
returntype = 'geographies'
searchtype = 'addressbatch'
url = f'https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}'

# establishing the file path and opening it
file_path = './data/address_batch_1.csv'
files = {'addressFile': (file_path, open(file_path, 'rb'), 'text/csv')}

# identifying the data type to pull
payload = {'benchmark':'Public_AR_Current', 'vintage':'Census2010_Current'}

# post the requests
s = requests.get(url, files=files, data=payload)

In [ ]:
s.status_code

400

In [37]:
mac10

7951020

### trying 10000 with vintage current current

In [15]:
# setting url information
returntype = 'geographies'
searchtype = 'addressbatch'
url = f'https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}'

# establishing the 
file_path = './data/address_batch_1.csv'
files = {'addressFile': (file_path, open(file_path, 'rb'), 'text/csv')}


payload = {'benchmark':'Public_AR_Current', 'vintage':'Current_Current'}

s = requests.post(url, files=files, data=payload)



ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

##### Notes:
* While it is frustrating, but it seems the most we can do for batch addresses wihtout a conneciton timeout is 2500 so I will use that number ot create a loop
* Both the api and the geocoding website do not actually support a 10000 file geocoding request

##### Actions:
* Try to get the geocoding through geocoder from python -  *Failed*
* Complete the tutorial with only 2500 addresses in the csv 
* Create a loop to complete the tutorial with all the data
* Switch to the geocoder 

## Geocoder Python Package
Failure

In [44]:
full_address_mac10 = unique_address_mac_10['incident_address'] +','+ unique_address_mac_10['city'] +','+ unique_address_mac_10['state'] +','+ unique_address_mac_10['zip'].astype(str)

type(full_address_mac10)

full_address_mac10.iloc[1]

'11506 LULLSTREAM DR,INDIANAPOLIS,IN,46229'

In [54]:
import geocoder
g = geocoder.google('Mountain View, CA')
g.latlng

## Complete tutorial

### Create for loop to save all the batches of address

In [76]:
# there will be 104 requests and 104 csv files
print(len(list(range(0,len(unique_address_mac_10)+1, 1000))))

ls = list(range(0,len(unique_address_mac_10)+1, 1000))

for i, v in enumerate(ls):
    # print(i, v)
    if i == 0:
        pass
    else:
        # stablish file path with i as the changed number for the addresses 
        file_path = f'./data/address_batch10_{i}.csv'
        
        batch = unique_address_mac_10.iloc[ls[i-1]:ls[i]]

        batch.to_csv(file_path, header=False)

260


### API request

In [77]:
returntype = 'geographies'
searchtype = 'addressbatch'
url = f'https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}'

# establishing the 
file_path = './data/address_batch_1.csv'
files = {'addressFile': (file_path, open(file_path, 'rb'), 'text/csv')}


payload = {'benchmark':'Public_AR_Current', 'vintage':'Census2010_Current'}

s = requests.post(url, files=files, data=payload)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
type(s)

In [ ]:
pd.read_csv(io.StringIO(s.text), sep=',', header=None, quoting=csv.QUOTE_ALL)

with pd.option_context(
    'display.width', None, 
    'display.max_columns', None,
    'display.max_colwidth', -1,
    'display.colheader_justify', 'left'
):
    print(df)